In [2]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
import os
drive_folder = '/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main'

os.chdir(drive_folder)

In [5]:
import zipfile

# Specify the path to the zip file and the destination directory
zip_file_path = '/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/archive.zip'
destination_directory = '/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_directory)


In [6]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [14]:
# Preprocess all trsin images
train_generator = train_data_gen.flow_from_directory(
        '/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/archive/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [15]:
# Preprocess all test images
validation_generator = validation_data_gen.flow_from_directory(
        '/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/archive/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam

# Create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

# Compile the model using learning_rate instead of decay
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


In [ ]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

<ipython-input-18-ae419f95bfec>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 511s 1s/step - loss: 1.8002 - accuracy: 0.2603 - val_loss: 1.6999 - val_accuracy: 0.3371
Epoch 2/50
448/448 [==============================] - 521s 1s/step - loss: 1.6141 - accuracy: 0.3754 - val_loss: 1.5146 - val_accuracy: 0.4233
Epoch 3/50
448/448 [==============================] - 516s 1s/step - loss: 1.5194 - accuracy: 0.4165 - val_loss: 1.4545 - val_accuracy: 0.4428
Epoch 4/50
448/448 [==============================] - 518s 1s/step - loss: 1.4469 - accuracy: 0.4462 - val_loss: 1.3904 - val_accuracy: 0.4674
Epoch 5/50
448/448 [==============================] - 506s 1s/step - loss: 1.3891 - accuracy: 0.4680 - val_loss: 1.3474 - val_accuracy: 0.4851
Epoch 6/50
144/448 [========>.....................] - ETA: 5:28 - loss: 1.3490 - accuracy: 0.4876

In [ ]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/model/emotion_model.json", "w") as json_file:
    json_file.write(model_json)


In [ ]:
# save trained model weight in .h5 file
emotion_model.save_weights('/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/model/emotion_model.h5')